In [3]:
from __future__ import absolute_import
import sys
sys.path.append("..")
import numpy as np
np.seterr(invalid='raise')
import pandas as pd
import datetime
import csv
import os
import uuid
import matplotlib.pyplot as plt
from utils.data import read_h5_fx_history, read_csv_history
from utils.globals import EPS, DATASETS_DIR, OUTPUTS_DIR, CAPITAL_BASE_MULTIPLIER, MAX_WEIGHT, RISK

In [20]:
instruments = ['EUR/USD', 'USD/JPY', 'AUD/USD', 'USD/CAD', 'GBP/USD', 'NZD/USD', 'GBP/JPY', 'EUR/JPY', 'AUD/JPY', 'EUR/GBP', 'USD/CHF']

current_prices = [1.10851,109.81900,0.68445,1.30728,1.30431,0.65952,143.24450,121.72950,75.17300,0.84978,0.96874 ]

##for standard lot
expected_values = [10.00, 9.10, 10.00, 7.65, 10.00, 10.00, 9.10, 9.10, 9.10, 13.05, 10.32]

In [14]:
def _calculate_pip_value_in_account_currency(currency, current_prices):
        
        pip_values = []
        
        #dictionary to keep prices for each currency, assuming that current_prices has prices in the same order as instruments list has instument names
        prices_for_currency = {}
        
        instrument_index = 0
        for instrument in instruments:
            prices_for_currency[instrument] = current_prices[instrument_index]
            instrument_index += 1
            
            
        #account currencu is USD
        if currency == 'USD':
            m = 0            
            for instrument in instruments:                                               
                first_currency = instrument[0:3]
                second_currency = instrument[4:7]
                                
                #counter currency same as account currency
                if second_currency == 'USD':
                    pip_value = 0.0001
                #base currency same as account currency    
                elif first_currency == 'USD':
                    #counter currency is not JPY
                    if second_currency != 'JPY':
                        pip_value = 0.0001/current_prices[m]
                    #counter currency is JPY
                    else: pip_value = 0.01/current_prices[m] 
                #none of the currency pair is the same as account currency
                #is needed the currency rate for the base currency/account currency
                else:
                    ##base currency/account currency rate is retrieved from stored values in dictionary
                    base_account_rate = prices_for_currency[first_currency+"/"+currency]
                    
                    if second_currency == 'JPY':
                        pip_value = base_account_rate * 0.01/current_prices[m]
                    else: pip_value = base_account_rate * 0.0001/current_prices[m] 
                        
                pip_values.append(pip_value * 100000)
                m += 1    

      
        return pip_values 


In [21]:
pip_values_calculated = _calculate_pip_value_in_account_currency('USD', current_prices)

print(expected_values)
print(pip_values_calculated)

[10.0, 9.1, 10.0, 7.65, 10.0, 10.0, 9.1, 9.1, 9.1, 13.05, 10.32]
[10.0, 9.105892422986914, 10.0, 7.6494706566305615, 10.0, 10.0, 9.10548048965231, 9.106338233542404, 9.104997805062988, 13.04467038527619, 10.322687201932407]
